# Assessment Sebastian Schmitt 40409930

# SVM classifier

## Load the JSON files 
Import the json files and store them in arrays.
As we have more than one record in our json file, we can't use json.loads().
Then we would receive an JSONDecodeError. 
Therefore we read each object in the JSON file and then append it to an array with json.loads(line).

In [41]:
import json
# has to be global variable / array
neutral_tweets = []
sexist_tweets = []
racist_tweets = []
for line in open ('neither.json', 'r'):
    neutral_tweets.append(json.loads(line))
    
for line in open ('sexism.json', 'r'):
    sexist_tweets.append(json.loads(line))
    
for line in open ('racism.json', 'r'):
    racist_tweets.append(json.loads(line))

## Convert to data-frame
Afterwards we import pandas and convert the arrays into data-frames. This gives better readability od the data in the json files.

In [42]:
import pandas

In [43]:
neutral_tweets = pandas.DataFrame(neutral_tweets)
sexist_tweets = pandas.DataFrame(sexist_tweets)
racist_tweets = pandas.DataFrame(racist_tweets)

In [44]:
neutral_tweets = neutral_tweets[['text']]
sexist_tweets = sexist_tweets[['text']]
racist_tweets = racist_tweets[['text']]

## Convert back to list
It is easier to iterate through lists (or I havent figured out how to do it properly with dataframes)

In [45]:
neutral_tweets = neutral_tweets['text'].values.tolist()
sexist_tweets = sexist_tweets['text'].values.tolist()
racist_tweets = racist_tweets['text'].values.tolist()

In [46]:
def add_to_all_tweet(list, label):
    for tweet in list:
        word = label
        word += tweet
        all_tweets.append(word)

Combine the three lists and also add a number as a label

In [47]:
all_tweets=[]
add_to_all_tweet(neutral_tweets, "0")
add_to_all_tweet(sexist_tweets, "1")
add_to_all_tweet(racist_tweets, "2")

In [48]:
all_tweets[:5]

['0Someone is going home #mkr ...that obviously cannot cook !',
 "0They didn't even wash the chicken 😩 #MKR",
 '0#MKR Is honestly so fucking staged. The most over rated show after #ImACelebrityAU #MKR2015',
 "0Can someone smash that bottle of Rose &amp; Lime Cordial over Drasko's head please #MKR",
 '0Will someone pls assist Colin in the washing of his hair. Sorry Colin, big fan but pls... Some shampoo mate! #mkr']

Split the tweets in training and test data

In [49]:
import random

def split_data(data):
    random.shuffle(data)
    split_point = int((len(data)/4))
    return (all_tweets[split_point:], all_tweets[:split_point])

train_data, test_data = split_data(all_tweets)

Create lists with the labels and the text (in this case the feature) for each the test and the training data

In [50]:
def preprocess_data_SVM(data):
    labels = []
    reviews = []
    for tweet in data:
        labels.append(tweet[0])
        reviews.append(tweet[1:])
    return (labels,reviews)

train_labels, train_reviews = preprocess_data_SVM(train_data)
test_labels, test_reviews = preprocess_data_SVM(test_data)

Feature Extraction/Vectorisation

In [51]:
from sklearn.feature_extraction.text import TfidfVectorizer

In [52]:
vectorizer = TfidfVectorizer(min_df=5,sublinear_tf=True)
train_vectors = vectorizer.fit_transform(train_reviews)
test_vectors = vectorizer.transform(test_reviews)

In [53]:
print(vectorizer.get_feature_names())

['000', '0xabad1dea', '0xjared', '10', '100', '1000', '11', '12', '120', '13', '14', '1400', '15', '17', '18', '1life0continues', '1st', '20', '200', '2014', '2015', '2016', '21', '22', '23', '24', '25', '27', '28', '280', '2nd', '30', '33', '34', '3rd', '40', '400', '4th', '50', '60', '600', '80', '800', '8bitbecca', '8chan', '8cmfoozwxz', '90', '95', '98halima', '99', '_blackglass__', '_dirtytruths', '_enzk', '_median_', '_roguest4r', '_watch_observe_', 'a_man_in_black', 'aalwuhaib1977', 'abbswinston', 'ability', 'able', 'abortion', 'about', 'above', 'absolute', 'absolutely', 'absurd', 'abt', 'abu', 'abu__ramadi', 'abu_baraa1', 'abunaseeha2', 'abuse', 'abused', 'abusive', 'abuyigitalcahrz', 'abyad', 'ac360', 'accent', 'accept', 'acceptable', 'accepted', 'accepting', 'according', 'account', 'accounts', 'acct', 'accurate', 'accused', 'across', 'act', 'acting', 'action', 'actions', 'actively', 'activities', 'actual', 'actually', 'ad', 'adam', 'add', 'added', 'address', 'admit', 'adnansa

## Classifier
Set the classifier to SVM and train the classifier with the training data.

In [54]:
from sklearn import svm

#SVM Linear Kernel 
classifier_linear = svm.SVC(kernel='linear')
classifier_linear.fit(train_vectors, train_labels)

SVC(C=1.0, cache_size=200, class_weight=None, coef0=0.0,
  decision_function_shape='ovr', degree=3, gamma='auto_deprecated',
  kernel='linear', max_iter=-1, probability=False, random_state=None,
  shrinking=True, tol=0.001, verbose=False)

Predict the test data

In [55]:
prediction_linear = classifier_linear.predict(test_vectors)

In [56]:
import warnings
from sklearn.metrics import classification_report

target_names = ['neutral', 'sexist', 'racist']
warnings.filterwarnings('ignore')
report_linear = classification_report(test_labels, prediction_linear, target_names=target_names)

In [57]:
print(report_linear)

              precision    recall  f1-score   support

     neutral       0.85      0.94      0.89      2867
      sexist       0.86      0.64      0.73       849
      racist       0.78      0.69      0.74       510

   micro avg       0.85      0.85      0.85      4226
   macro avg       0.83      0.76      0.79      4226
weighted avg       0.85      0.85      0.84      4226



# Decision trees

## Load the JSON files 
Fresh load

In [58]:
import json
# has to be global variable / array
from numpy import median
neutral_tweets = []
sexist_tweets = []
racist_tweets = []
for line in open ('neither.json', 'r'):
    neutral_tweets.append(json.loads(line))
    
for line in open ('sexism.json', 'r'):
    sexist_tweets.append(json.loads(line))
    
for line in open ('racism.json', 'r'):
    racist_tweets.append(json.loads(line))

Convert to dataframes to have a better overview and make it easier to delete unwanted columns

In [59]:
import pandas
neutral_tweets = pandas.DataFrame(neutral_tweets)
sexist_tweets = pandas.DataFrame(sexist_tweets)
racist_tweets = pandas.DataFrame(racist_tweets)

Only keep the columns for favourites, retweets and the text of the tweets

In [60]:
neutral_tweets = neutral_tweets[['favorite_count','retweet_count','text']]
sexist_tweets = sexist_tweets[['favorite_count','retweet_count','text']]
racist_tweets = racist_tweets[['favorite_count','retweet_count','text']]

## Convert back to list
It is easier to iterate through lists (or I havent figured out how to do it properly with dataframes)

In [61]:
neutral_tweets = neutral_tweets.values.tolist()
sexist_tweets = sexist_tweets.values.tolist()
racist_tweets = racist_tweets.values.tolist()

Methods for adding the word counts and labels

In [62]:
def add_word_count (list):
    for entry in list:
        text_of_tweet = entry[2]
        entry.append(len(text_of_tweet.split()))

def add_label (list, label):
    for entry in list:
        entry.append(label)

In [63]:
add_word_count(neutral_tweets)
add_word_count(sexist_tweets)
add_word_count(racist_tweets)
add_label(neutral_tweets, 'neutral')
add_label(sexist_tweets, 'sexist')
add_label(racist_tweets, 'racist')

Method for deleting the text

In [64]:
def delete_text (list):
    for entry in list:
        del entry[2]

In [65]:
delete_text(neutral_tweets)
delete_text(racist_tweets)
delete_text(sexist_tweets)

Format:
Each row is a tweet.
The last column is the label ('neutral', 'sexist' or 'racist')
The other columns are the labels (favourites, retweets and the word count)

In [66]:
header = ["favourites", "retweets", "wordcount", "label"]

Split the data into training and test data

In [67]:
import random

split_neutral = int(len(neutral_tweets)*0.75)
split_sexist = int(len(sexist_tweets)*0.75)
split_racist = int(len(racist_tweets)*0.75)
random.shuffle(neutral_tweets)
random.shuffle(sexist_tweets)
random.shuffle(racist_tweets)

training_tweets = neutral_tweets[:split_neutral] + sexist_tweets[:split_sexist] + racist_tweets[:split_racist] 
test_tweets = neutral_tweets[split_neutral:] + sexist_tweets[split_sexist:] + racist_tweets[split_racist:] 

Method to get all values of one column (e.g. the favourite counts)

In [68]:
def get_column(list, index):
    help = []
    for entry in list: 
        help.append(entry[index])
    return help

Calculate the average values for each column

In [69]:
import numpy
mean_favourites = numpy.mean(get_column(training_tweets, 0))
mean_retweets = numpy.mean(get_column(training_tweets, 1))
mean_word_count = numpy.mean(get_column(training_tweets, 2))

Method gives back the unique values for the rows and columns

In [70]:
def unique_values(rows, col):
    return set([row[col] for row in rows])

Gives back the count of the different classes (neutral, sexist and racist)

In [71]:
def class_counts(rows):
    counts = {}  
    for row in rows:
        label = row[-1]
        if label not in counts:
            counts[label] = 0
        counts[label] += 1
    return counts

This class is used to partition a dataset.
This class just records a column number (e.g., 0 for favourites) and a column value. The "match" method compares  a feature value with the feature value stored in the question. This helps to "get through" the decision tree.

In [72]:
class Question:
    def __init__(self, column, value):
        self.column = column
        self.value = value

    def match(self, example):
        # Compare the feature value in an example to the
        # feature value in this question.
        val = example[self.column]
        return val >= self.value

    def __repr__(self):
        # This is just a helper method to print
        # the question in a readable format.
        condition = ">="
        return "Is %s %s %s?" % (
            header[self.column], condition, str(self.value))
            

This method divides the dataset. For each row in the dataset, check if it matches (is bigger than) the question. If it is then it adds it to true rows, otherwise to false rows.

In [73]:
def partition(rows, question):
    
    true_rows, false_rows = [], []
    for row in rows:
        if question.match(row):
            true_rows.append(row)
        else:
            false_rows.append(row)
    return true_rows, false_rows


Calculates the Gini impurity for several tweets.

In [74]:
def gini(rows):
    counts = class_counts(rows)
    impurity = 1
    for lbl in counts:
        prob_of_lbl = counts[lbl] / float(len(rows))
        impurity -= prob_of_lbl**2
    return impurity

Calculates the info gain with the help of the current uncertainty.

In [75]:
def info_gain(left, right, current_uncertainty):
    p = float(len(left)) / (len(left) + len(right))
    return current_uncertainty - p * gini(left) - (1 - p) * gini(right)

In [76]:
current_uncertainty = gini(training_tweets)
current_uncertainty

0.48243552828270836

In [77]:
# Info gain when partitioning on mean of favourites
true_rows, false_rows = partition(training_tweets, Question(0, mean_favourites))
info_gain(true_rows, false_rows, current_uncertainty)

0.010069193025531331

In [78]:
# Info gain when partitioning on mean of retweets
true_rows, false_rows = partition(training_tweets, Question(1, mean_retweets))
info_gain(true_rows, false_rows, current_uncertainty)

0.0013059819220406221

In [79]:
# Info gain whenpartitioning on mean of the tweet word count
true_rows, false_rows = partition(training_tweets, Question(2, mean_word_count))
info_gain(true_rows, false_rows, current_uncertainty)

0.011644889321568735

Finds the best question to ask

In [81]:
def find_best_question(rows):
    best_information_gain = 0  
    best_question = None  
    current_uncertainty = gini(rows)
    number_columns = len(rows[0]) - 1  # number of columns

    for col in range(number_columns):  # for each feature

        values = set([row[col] for row in rows])  # unique values in the column

        for val in values:  # for each value

            question = Question(col, val)

            # try splitting the dataset
            true_rows, false_rows = partition(rows, question)

            # Skip this split if it doesn't divide the dataset.
            if len(true_rows) == 0 or len(false_rows) == 0:
                continue

            # Calculate the information gain from this split
            gain = info_gain(true_rows, false_rows, current_uncertainty)

            if gain >= best_information_gain:
                best_information_gain, best_question = gain, question

    return best_information_gain, best_question

In [82]:
best_information_gain, best_question = find_best_question(training_tweets)
best_question

Is wordcount >= 12?

In [83]:
# Info gain for the best question
true_rows, false_rows = partition(training_tweets, Question(2, 12))
info_gain(true_rows, false_rows, current_uncertainty)

0.013977326262631568

Class for a leaf

In [85]:
class Leaf:
    def __init__(self, rows):
        self.predictions = class_counts(rows)

Asks a question and holds a reference to the question, and to the two child nodes.

In [86]:
class Decision_Node:
    def __init__(self,
                 question,
                 true_branch,
                 false_branch):
        self.question = question
        self.true_branch = true_branch
        self.false_branch = false_branch

In [87]:
def build_tree(rows):
    # Try partitioing the dataset on each of the unique attributes,
    # calculate the information gain,
    # and return the question that produces the highest gain.
    gain, question = find_best_question(rows)

    # Base case: no further info gain
    # we'll return a leaf.
    if gain == 0:
        return Leaf(rows)

    # If we reach here, we have found a useful feature / value
    # to partition on.
    true_rows, false_rows = partition(rows, question)

    # Recursively build the true branch.
    true_branch = build_tree(true_rows)

    # Recursively build the false branch.
    false_branch = build_tree(false_rows)

    # Return a Question node.
    # This records the best feature / value to ask at this point,
    # as well as the branches to follow
    # dependingo on the answer.
    return Decision_Node(question, true_branch, false_branch)

In [88]:
def print_tree(node, spacing=""):
    
    # Base case: we've reached a leaf
    if isinstance(node, Leaf):
        print (spacing + "Predict", node.predictions)
        return

    # Print the question at this node
    print (spacing + str(node.question))

    # Call this function recursively on the true branch
    print (spacing + '--> True:')
    print_tree(node.true_branch, spacing + "  ")

    # Call this function recursively on the false branch
    print (spacing + '--> False:')
    print_tree(node.false_branch, spacing + "  ")

In [89]:
my_tree = build_tree(training_tweets)

In [90]:
print_tree(my_tree)

Is wordcount >= 12?
--> True:
  Is favourites >= 2?
  --> True:
    Is favourites >= 6?
    --> True:
      Is favourites >= 10?
      --> True:
        Is wordcount >= 17?
        --> True:
          Is retweets >= 24?
          --> True:
            Is retweets >= 26?
            --> True:
              Is favourites >= 57?
              --> True:
                Is favourites >= 58?
                --> True:
                  Predict {'neutral': 11}
                --> False:
                  Predict {'sexist': 1}
              --> False:
                Predict {'neutral': 19}
            --> False:
              Is favourites >= 42?
              --> True:
                Predict {'neutral': 1}
              --> False:
                Predict {'sexist': 3}
          --> False:
            Is favourites >= 62?
            --> True:
              Is favourites >= 63?
              --> True:
                Predict {'neutral': 11}
              --> False:
                Predict {'s

                                --> True:
                                  Predict {'neutral': 2, 'sexist': 1}
                                --> False:
                                  Predict {'neutral': 1, 'racist': 1}
                              --> False:
                                Predict {'neutral': 1, 'sexist': 1}
                        --> False:
                          Is retweets >= 12?
                          --> True:
                            Predict {'neutral': 5}
                          --> False:
                            Is retweets >= 11?
                            --> True:
                              Predict {'racist': 1}
                            --> False:
                              Predict {'neutral': 5}
                      --> False:
                        Is retweets >= 13?
                        --> True:
                          Is wordcount >= 16?
                          --> True:
                            Predict {'sex

In [91]:
def classify(row, node):

    # If we reach a leaf, return a prediction
    if isinstance(node, Leaf):
        return node.predictions

    # Decide whether to follow the true-branch or the false-branch.
    if node.question.match(row):
        return classify(row, node.true_branch)
    else:
        return classify(row, node.false_branch)

In [92]:
training_tweets[0]

[0, 0, 8, 'neutral']

In [93]:
# Predicts the first tweets as neutral with a confidence of 6
classify(training_tweets[0], my_tree)

{'neutral': 170, 'sexist': 29, 'racist': 9}

In [94]:
training_tweets[-1]

[0, 0, 17, 'racist']

In [95]:
classify(training_tweets[-1], my_tree)

{'neutral': 161, 'sexist': 42, 'racist': 124}

In [96]:
def print_leaf(counts):
    total = sum(counts.values()) * 1.0
    probs = {}
    for lbl in counts.keys():
        probs[lbl] = str(int(counts[lbl] / total * 100)) + "%"
    return probs

In [97]:
print_leaf(classify(training_tweets[0], my_tree))

{'neutral': '81%', 'sexist': '13%', 'racist': '4%'}

Overview about the predictions

In [98]:
for row in test_tweets:
    print ("Actual: %s. Predicted: %s" %
           (row[-1], print_leaf(classify(row, my_tree))))

Actual: neutral. Predicted: {'neutral': '56%', 'sexist': '13%', 'racist': '29%'}
Actual: neutral. Predicted: {'neutral': '83%', 'sexist': '15%', 'racist': '1%'}
Actual: neutral. Predicted: {'neutral': '49%', 'sexist': '12%', 'racist': '37%'}
Actual: neutral. Predicted: {'neutral': '60%', 'sexist': '20%', 'racist': '18%'}
Actual: neutral. Predicted: {'neutral': '70%', 'sexist': '25%', 'racist': '3%'}
Actual: neutral. Predicted: {'neutral': '66%', 'sexist': '33%'}
Actual: neutral. Predicted: {'neutral': '75%', 'sexist': '25%'}
Actual: neutral. Predicted: {'racist': '100%'}
Actual: neutral. Predicted: {'neutral': '73%', 'sexist': '17%', 'racist': '8%'}
Actual: neutral. Predicted: {'neutral': '77%', 'sexist': '17%', 'racist': '5%'}
Actual: neutral. Predicted: {'neutral': '100%'}
Actual: neutral. Predicted: {'neutral': '60%', 'sexist': '40%'}
Actual: neutral. Predicted: {'neutral': '60%', 'sexist': '20%', 'racist': '18%'}
Actual: neutral. Predicted: {'neutral': '100%'}
Actual: neutral. Pred

Actual: neutral. Predicted: {'neutral': '100%'}
Actual: neutral. Predicted: {'neutral': '100%'}
Actual: neutral. Predicted: {'neutral': '62%', 'sexist': '13%', 'racist': '23%'}
Actual: neutral. Predicted: {'neutral': '50%', 'sexist': '15%', 'racist': '34%'}
Actual: neutral. Predicted: {'neutral': '71%', 'sexist': '28%'}
Actual: neutral. Predicted: {'neutral': '67%', 'sexist': '23%', 'racist': '9%'}
Actual: neutral. Predicted: {'neutral': '57%', 'sexist': '34%', 'racist': '8%'}
Actual: neutral. Predicted: {'neutral': '77%', 'sexist': '17%', 'racist': '5%'}
Actual: neutral. Predicted: {'racist': '100%'}
Actual: neutral. Predicted: {'neutral': '60%', 'sexist': '20%', 'racist': '18%'}
Actual: neutral. Predicted: {'neutral': '50%', 'sexist': '50%'}
Actual: neutral. Predicted: {'neutral': '60%', 'sexist': '20%', 'racist': '18%'}
Actual: neutral. Predicted: {'neutral': '47%', 'sexist': '12%', 'racist': '39%'}
Actual: neutral. Predicted: {'neutral': '62%', 'sexist': '29%', 'racist': '8%'}
Actu

In [99]:
for row in test_tweets:
    dict = classify(row, my_tree)
    neutral = dict.get('neutral')
    if neutral is None:
        neutral = 0
    sexist = dict.get('sexist')
    if sexist is None:
        sexist = 0
    racist = dict.get('racist')
    if racist is None:
        racist = 0
    if (neutral > sexist and neutral > racist):
        print ("Actual: %s. Predicted: %s" % (row[-1], 'neutral'))
    elif (sexist > neutral and sexist > racist):
        print ("Actual: %s. Predicted: %s" % (row[-1], 'sexist'))
    elif (racist > neutral and racist > sexist):
        print ("Actual: %s. Predicted: %s" %
           (row[-1], 'racist'))

Actual: neutral. Predicted: neutral
Actual: neutral. Predicted: neutral
Actual: neutral. Predicted: neutral
Actual: neutral. Predicted: neutral
Actual: neutral. Predicted: neutral
Actual: neutral. Predicted: neutral
Actual: neutral. Predicted: neutral
Actual: neutral. Predicted: racist
Actual: neutral. Predicted: neutral
Actual: neutral. Predicted: neutral
Actual: neutral. Predicted: neutral
Actual: neutral. Predicted: neutral
Actual: neutral. Predicted: neutral
Actual: neutral. Predicted: neutral
Actual: neutral. Predicted: neutral
Actual: neutral. Predicted: sexist
Actual: neutral. Predicted: neutral
Actual: neutral. Predicted: neutral
Actual: neutral. Predicted: sexist
Actual: neutral. Predicted: neutral
Actual: neutral. Predicted: neutral
Actual: neutral. Predicted: neutral
Actual: neutral. Predicted: neutral
Actual: neutral. Predicted: neutral
Actual: neutral. Predicted: neutral
Actual: neutral. Predicted: neutral
Actual: neutral. Predicted: neutral
Actual: neutral. Predicted: neu

Actual: neutral. Predicted: neutral
Actual: neutral. Predicted: neutral
Actual: neutral. Predicted: neutral
Actual: neutral. Predicted: neutral
Actual: neutral. Predicted: neutral
Actual: neutral. Predicted: neutral
Actual: neutral. Predicted: neutral
Actual: neutral. Predicted: neutral
Actual: neutral. Predicted: neutral
Actual: neutral. Predicted: sexist
Actual: neutral. Predicted: neutral
Actual: neutral. Predicted: neutral
Actual: neutral. Predicted: neutral
Actual: neutral. Predicted: neutral
Actual: neutral. Predicted: neutral
Actual: neutral. Predicted: neutral
Actual: neutral. Predicted: neutral
Actual: neutral. Predicted: neutral
Actual: neutral. Predicted: neutral
Actual: neutral. Predicted: neutral
Actual: neutral. Predicted: neutral
Actual: neutral. Predicted: sexist
Actual: neutral. Predicted: neutral
Actual: neutral. Predicted: neutral
Actual: neutral. Predicted: neutral
Actual: neutral. Predicted: neutral
Actual: neutral. Predicted: neutral
Actual: neutral. Predicted: ne

Actual: neutral. Predicted: neutral
Actual: neutral. Predicted: neutral
Actual: neutral. Predicted: neutral
Actual: neutral. Predicted: neutral
Actual: neutral. Predicted: neutral
Actual: neutral. Predicted: neutral
Actual: neutral. Predicted: neutral
Actual: neutral. Predicted: neutral
Actual: neutral. Predicted: neutral
Actual: neutral. Predicted: neutral
Actual: neutral. Predicted: neutral
Actual: neutral. Predicted: neutral
Actual: neutral. Predicted: neutral
Actual: neutral. Predicted: neutral
Actual: neutral. Predicted: neutral
Actual: neutral. Predicted: neutral
Actual: neutral. Predicted: neutral
Actual: neutral. Predicted: neutral
Actual: neutral. Predicted: neutral
Actual: neutral. Predicted: neutral
Actual: neutral. Predicted: neutral
Actual: neutral. Predicted: neutral
Actual: neutral. Predicted: neutral
Actual: neutral. Predicted: neutral
Actual: neutral. Predicted: neutral
Actual: neutral. Predicted: neutral
Actual: neutral. Predicted: neutral
Actual: neutral. Predicted: 

Create a list with the real class and the class with the highest prediction for each tweet. Also count the right and wrong predictions

In [100]:
list = []
right_prediction = 0
wrong_prediction = 0
for row in test_tweets:
    row_list = []
    real_value = row[-1]
    row_list.append(real_value)
    dict = classify(row, my_tree)
    neutral = dict.get('neutral')
    if neutral is None:
        neutral = 0
    sexist = dict.get('sexist')
    if sexist is None:
        sexist = 0
    racist = dict.get('racist')
    if racist is None:
        racist = 0
    if (neutral > sexist and neutral > racist) or (neutral >= sexist and neutral > racist) or (neutral > sexist and neutral >= racist):
        row_list.append('neutral')
        predicted_value = 'neutral'
    elif (sexist > neutral and sexist > racist) or (sexist >= neutral and sexist > racist) or (sexist > neutral and sexist >= racist):
        row_list.append('sexist')
        predicted_value = 'sexist'
    elif (racist > neutral and racist > sexist) or (racist >= neutral and racist > sexist) or (racist > neutral and racist >= sexist):
        row_list.append('racist')
        predicted_value = 'racist'
    list.append(row_list)
    if (real_value == predicted_value):
        right_prediction = right_prediction + 1 
    else:
        wrong_prediction = wrong_prediction + 1

In [101]:
list[:5]

[['neutral', 'neutral'],
 ['neutral', 'neutral'],
 ['neutral', 'neutral'],
 ['neutral', 'neutral'],
 ['neutral', 'neutral']]

In [102]:
right_prediction

2837

In [103]:
wrong_prediction

1391

## Evaluation

In [104]:
var = right_prediction / (right_prediction + wrong_prediction)
var

0.6710028382213813

The actual tweet counts and the predicted counts

In [105]:
counts = {}  
for row in list:
    label = row[-1]
    if label not in counts:
        counts[label] = 0
    counts[label] += 1
print("Precicted tweet counts: " + str(counts))

Precicted tweet counts: {'neutral': 3850, 'racist': 23, 'sexist': 355}


In [106]:
print("Actual tweet counts: " + str(class_counts(test_tweets)))

Actual tweet counts: {'neutral': 2876, 'sexist': 858, 'racist': 494}


In [107]:
# precision = true positive / true positive + false positive
# recall = true positive / true positive + false negative
# predicted as neutral when actual not neutral false_positive++
# predicted as neutral when actual neutral true_positive++
# predicted as not neutral wehen actual neutral false_negative++
def precision (label):
    true_positive = 0
    false_positive = 0
    for row in list:
        if (row[-1] == label): #predicted class
            if row[0] == label: #actual class
                true_positive = true_positive +1
            else:
                false_positive = false_positive + 1
    return (true_positive/(true_positive + false_positive)) 

In [108]:
def recall (label):
    true_positive = 0
    false_positive = 0
    false_negative = 0
    for row in list:
        if (row[0] == label): #actual class
            if row[-1] == label: #predicted class
                true_positive = true_positive +1
            else:
                false_negative = false_negative + 1
    return (true_positive/(true_positive + false_negative)) 

In [109]:
def f1_score (label):
    return 2*((precision(label)*recall(label))/((precision(label)+recall(label))))

Print out the results

In [110]:
from tabulate import tabulate
print(tabulate([['neutral', precision("neutral"), recall("neutral"), f1_score("neutral")], 
                ['sexist', precision("sexist"), recall("sexist"), f1_score("sexist")] , 
               ['racist', precision("racist"), recall("racist"), f1_score("racist")]],
              ['class', "precision", "recall", "f1_score"]))

class      precision      recall    f1_score
-------  -----------  ----------  ----------
neutral     0.696364  0.932197     0.797205
sexist      0.43662   0.180653     0.255565
racist      0.173913  0.00809717   0.0154739
